In [23]:
import osmnx as ox
import networkx as nx
import numpy as np
import matplotlib.cm as cm

In [3]:
location="Trento, Territorio Val d'Adige, Provincia di Trento, Trentino-Alto Adige/Südtirol, Italy"

In [5]:
# download Trento 
G = ox.graph_from_place(location, network_type='bike')

In [6]:
# calculate the metrics for each arc
G = ox.add_edge_bearings(ox.get_undirected(G))
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

In [10]:
weights = {'primary': 1, 'secondary': 0.8, 'tertiary': 0.6, 'residential': 0.4, 'unclassified': 0.2, 'cycleway': 1.2}

In [22]:
# Calcola il BikeScore per ogni strada nell'area di interesse
bikescore = {}
for u, v, k, data in G.edges(keys=True, data=True):
    if 'cycleway' in data['highway']:
        # se la strada è una pista ciclabile, il suo peso viene moltiplicato per un fattore maggiore
        weight = weights['cycleway']
    else:
        # altrimenti, viene utilizzato il peso corrispondente al tipo di strada
        weight = weights.get(''.join(data['highway']), 0)
    slope = data.get('grade', 0)  # inclinazione della strada, se disponibile
    bikescore[(u, v, k)] = weight * (1 - np.tanh(slope))  # calcolo del punteggio per questa strada

In [24]:
# Visualizza una mappa colorata in base ai valori di BikeScore
cmap = cm.get_cmap('plasma')
norm = cm.colors.Normalize(vmin=min(bikescore.values()), vmax=max(bikescore.values()))
colors = [cmap(norm(bike)) for bike in bikescore.values()]
ox.plot_graph_folium(G, edge_color=colors, edge_width=2)